In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Change the directory to main folder
%cd /content/drive/MyDrive/Colab Notebooks/polynomial

/content/drive/MyDrive/Colab Notebooks/polynomial


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import time
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
data_frames = []
chunk_value = 5000
temp = pd.read_json('/content/drive/MyDrive/Colab Notebooks/polynomial/Cell_Phones_and_Accessories.json.gz', lines=True, chunksize=chunk_value)
for chunk in temp:
        data_frames.append(chunk)

df = pd.concat(data_frames, ignore_index=True, sort=True)

In [5]:
rev = []
for i in range(0,df.shape[0]):
  if df.iloc[i,4][7:11] == '2017':
    rev.append(i)
        
df = df.iloc[rev, :]
df = df.reset_index(drop=True)

In [6]:
df.head()

,asin,image,overall,reviewText,reviewTime,reviewerID,reviewerName,style,summary,unixReviewTime,verified,vote
0,7391002801,NaN,1,Had this for 2 weeks. Had to replace screen p...,"12 29, 2017",AN04BLRG7BD8I,J. Inman,{'Color:': ' Blue'},Outer ring very flimsy.,1514505600,True,NaN
1,7391002801,NaN,1,The apple is not centered in the hole on the b...,"12 26, 2017",A3PHYA8A965CYU,Morgan Epperson,{'Color:': ' Rose Gold'},"Pretty, but doesnt fit well.",1514246400,True,NaN
2,7391002801,[https://images-na.ssl-images-amazon.com/image...,1,Case is cheaply made. If you aren't using an a...,"12 24, 2017",A3N778P1L4YH9Y,McKenna Clark,{'Color:': ' Blue'},Case is cheaply made. If you aren't using an ...,1514073600,True,2.0
3,7391002801,NaN,5,This case is a really good thing. When you're ...,"12 14, 2017",A3PHJYND753HBC,Amazon Customer,{'Color:': ' Grey2'},"Very low price, unbelievably high quality",1513209600,True,NaN
4,7391002801,NaN,5,I use it for a long time and the look of the c...,"12 14, 2017",A1SUHZO4ONJ5YC,Amazon Customer,{'Color:': ' Grey2'},Premium quality!,1513209600,True,NaN


In [7]:
df.shape

(975705, 12)

In [8]:
df = df.sample(n = 100000)
df = df[['reviewText','overall']]

In [9]:
df.head()

,reviewText,overall
319647,Great,5
417073,Finally a car cell phone mount that works 100%...,5
334072,Fits like a glove and just non-slippery enough...,5
808983,"It fits well , but i thougt( woukd have prefer...",4
529268,Bought this brand of screen protector for my l...,5


In [10]:
data=df

data = data.replace(4,'positive')
data = data.replace(5,'positive')

data = data.replace(2,'negative')
data = data.replace(1,'negative')

data = data.replace(3,'neutral')


print(data.head())
data.shape

                                               reviewText   overall
319647                                              Great  positive
417073  Finally a car cell phone mount that works 100%...  positive
334072  Fits like a glove and just non-slippery enough...  positive
808983  It fits well , but i thougt( woukd have prefer...  positive
529268  Bought this brand of screen protector for my l...  positive


(100000, 2)

In [11]:
data = data.dropna()

In [12]:
data.head()

,reviewText,overall
319647,Great,positive
417073,Finally a car cell phone mount that works 100%...,positive
334072,Fits like a glove and just non-slippery enough...,positive
808983,"It fits well , but i thougt( woukd have prefer...",positive
529268,Bought this brand of screen protector for my l...,positive


In [13]:
print(len(data[data['overall'] == 'positive']))
print(len(data[data['overall'] == 'negative']))
print(len(data[data['overall'] == 'neutral']))

70826
21328
7736


In [14]:
# Data Balancing

d = []

pos_data = data[data['overall'] == 'positive']
d.append(pos_data)

neg_data = data[data['overall'] == 'negative']
d.append(neg_data)

neu_data = data[data['overall'] == 'neutral']
d.append(neu_data)

d = pd.concat(d)

In [15]:
import string
def punctuation_removal(s):
  mod_punch = [i for i in string.punctuation if i != '_']
  table = str.maketrans(dict.fromkeys(mod_punch))  
  new_s = s.translate(table)  
  new_s = new_s.replace('_',' ')
  return new_s.lower()


cleaned_text = []
for index, text in d['reviewText'].iteritems():
  cleaned_text.append(punctuation_removal(str(text)))

In [16]:
d.shape, len(cleaned_text)

((99890, 2), 99890)

In [17]:
d.insert(2, "cleaned_data", cleaned_text)

d.head()

,reviewText,overall,cleaned_data
319647,Great,positive,great
417073,Finally a car cell phone mount that works 100%...,positive,finally a car cell phone mount that works 100 ...
334072,Fits like a glove and just non-slippery enough...,positive,fits like a glove and just nonslippery enough ...
808983,"It fits well , but i thougt( woukd have prefer...",positive,it fits well but i thougt woukd have preferre...
529268,Bought this brand of screen protector for my l...,positive,bought this brand of screen protector for my l...


In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(d['cleaned_data'], 
                                                    d['overall'], 
                                                    test_size=0.25, 
                                                    random_state=0)

X_train = X_train.fillna(' ')
X_test = X_test.fillna(' ')

In [19]:
# Define the model and its parameters

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],    
             'tfidf__use_idf': (True, False), 
             } 
clf_linearSVC_pipe = Pipeline([("vect", CountVectorizer()),
                               ("tfidf", TfidfTransformer()), 
                               ("clf_linearSVC", LinearSVC())])

gs_clf_LinearSVC_pipe = GridSearchCV(clf_linearSVC_pipe, parameters, n_jobs=-1)

gs_clf_LinearSVC_pipe.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf_linearSVC', LinearSVC())]),
             n_jobs=-1,
             param_grid={'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

In [20]:
prediction = gs_clf_LinearSVC_pipe.predict(X_test)
report = classification_report(prediction, Y_test,output_dict=True )
print(classification_report(prediction, Y_test))

              precision    recall  f1-score   support

    negative       0.85      0.79      0.82      5796
     neutral       0.12      0.45      0.19       533
    positive       0.97      0.92      0.94     18644

    accuracy                           0.88     24973
   macro avg       0.65      0.72      0.65     24973
weighted avg       0.92      0.88      0.90     24973



In [21]:
print(confusion_matrix(Y_test,prediction))

[[ 4586   146   634]
 [  781   239   936]
 [  429   148 17074]]


In [22]:
accuracy_score(Y_test, prediction)*100

87.69070596243942

In [23]:
from joblib import dump, load
model_file = '/content/drive/MyDrive/Colab Notebooks/polynomial/final_model.joblib'
dump(gs_clf_LinearSVC_pipe, model_file) 

['/content/drive/MyDrive/Colab Notebooks/polynomial/final_model.joblib']